In [40]:
import keyring
import getpass
import github
import subprocess
import datetime
import pathlib

In [4]:
# access_token = getpass.getpass()
# keyring.set_password("github-access-token", "radiotherapyai-bot", access_token)

In [5]:
access_token = keyring.get_password("github-access-token", "radiotherapyai-bot")

In [6]:
github_instance = github.Github(access_token)

In [7]:
github_instance

In [8]:
for repo in github_instance.get_user().get_repos():
    print(repo.name)

regulatory


In [9]:
repo = github_instance.get_repo("RadiotherapyAI/regulatory")

In [10]:
repo.name

'regulatory'

In [11]:
pull_requests = list(repo.get_pulls(direction='asc'))

In [12]:
pr_with_an_approval = repo.get_pull(20)

In [13]:
reviews = list(pr_with_an_approval.get_reviews())

In [14]:
for review in reviews:
    if review.state == "APPROVED":
        print(f"{review.user.name} approved {review.commit_id} on the {review.submitted_at.strftime('%Y-%b-%d')}")

Oliver Eidel approved ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5 on the 2022-Mar-01


In [15]:
review = reviews[0]

In [16]:
review.commit_id

'ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5'

In [17]:
review.user

NamedUser(login="olieidel")

In [18]:
review.submitted_at.strftime('%Y-%b-%d')

'2022-Mar-01'

In [19]:
list()

[]

In [20]:
for a_file in pr_with_an_approval.get_files():    
    print(f"https://github.com/RadiotherapyAI/regulatory/blob/{review.commit_id}/{a_file.filename}")

https://github.com/RadiotherapyAI/regulatory/blob/ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5/src/documents/drafts/62304-mapping.md
https://github.com/RadiotherapyAI/regulatory/blob/ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5/src/documents/drafts/quality-manual-policy-objectives.md
https://github.com/RadiotherapyAI/regulatory/blob/ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5/src/documents/drafts/sop-integrated-software-development.md


In [21]:
pr_with_an_approval.merged_by.name

'Simon Biggs'

In [22]:
pr_with_an_approval.merged_at

datetime.datetime(2022, 3, 23, 5, 57, 58)

In [42]:
repo_root = pathlib.Path('../../').resolve()

In [37]:
path = './src/documents/released/sop-document-record-control/index.md'

In [51]:
result = subprocess.run([
    'git', 
    'log', 
    '--follow', 
    '--pretty=format:%H',
    '--all',
    '--',
    path
], stdout=subprocess.PIPE, cwd=repo_root)

In [ ]:
# Could collect all of the commits sha that cover all of the draft, production, and archive files
# Save the file path to commit history mapping
# Then get all the corresponding PRs for the unique commits via the GraphQL API in one request

In [55]:
followed_commits_for_file = result.stdout.decode().split("\n")
followed_commits_for_file

['5e57060965e4089b30812d3f79b63b3cffa8426f',
 '401bf64ed021d337f3824569ccb175444b69310b',
 '59949b3d91b57d3fb72ce8560e3df573d6612d80',
 'd13989a75082db70c8142b25f44627d0268e19a5',
 '5b7b941fdb4483b10875a86f3d07886686ff3fa2',
 '33ec2114f025576ce00cbbc5ad3b3ac544d31d03',
 '42794c3551203a45f0432b4a45de2df614f353d4',
 '7872e86cc3c0cf0d5a8ca8bbfc92a63af1be7d57',
 'fdc8f817470bba0560685f8478dbd81967c4290f',
 'cac08e953bcb92ae89d4a96e449e3d011ad73a10',
 '18ee0f83a3c944f7b113e9c30f37cf50717ca730',
 'a5ba0930d495b2c7a08b29f1d880c34dc8be05ad',
 'bcb4f64bf81b7a98a2e15508b8ebdbe3a17058c0',
 'e58b33f9cda722dea78243cd73293c66d01bfcca',
 'a46d571a1feb3d117fc29092583fd1eb80254428',
 '4f6253619d982af3902ed496919089c13141b33b',
 '885e2948632754423fbb705d05c398fc7b36e496',
 '83e9f0107eb29c1a70bb7f77c19f9af663193666',
 '0c4832f280522c85dc57c4b917ea9e3a9986b321',
 '965503ae655735aa32cd59879a880af94be1b32b',
 '3b3592ef5a5d59234e557075707fb036b59a993c',
 '9c8245c9df7e6872e1bf44d751dd47c2f381c4f8',
 '8d3a15dc

In [56]:
repo.get_commit(followed_commits_for_file[0])

Commit(sha="5e57060965e4089b30812d3f79b63b3cffa8426f")

In [26]:
# repo.get_commits?

In [27]:
commits = list(repo.get_commits(path=path))

In [28]:
commit = commits[0]

In [29]:
commit.committer.name

'GitHub Web Flow'

In [60]:
list(commit.get_statuses())

[]

In [61]:
commit.last_modified

'Fri, 04 Mar 2022 23:39:36 GMT'

In [58]:
def commit_date(commit):
    return datetime.datetime.strptime(commit.last_modified, "%a, %d %b %Y %H:%M:%S %Z")
    

In [59]:
commit_date(commit).strftime('%Y-%b-%d')

'2022-Mar-04'

In [60]:
author_and_timestamp = []
prs_for_this_file = set()

for commit_sha in followed_commits_for_file:
    commit = repo.get_commit(commit_sha)
    author_and_timestamp.append((commit.committer.name, commit_date(commit)))
    prs_for_this_file.update(commit.get_pulls())
    
    
prs_for_this_file

{PullRequest(title="Allow for PR only edits", number=18),
 PullRequest(title="Document record control", number=5),
 PullRequest(title="Record control refinements", number=9),
 PullRequest(title="Release sop drc, remove rdm templates for now", number=14),
 PullRequest(title="Request Oliver's review", number=13),
 PullRequest(title="Update document record control | Utilise PR review process", number=12),
 PullRequest(title="add in a codeowners file", number=8),
 PullRequest(title="delete sop-doc-record control for now", number=10),
 PullRequest(title="revert to original template", number=11),
 PullRequest(title="sign document creation with a gpg signed commit", number=7)}

In [ ]:
# Doesn't have the full history of the file.
# Potentially need to use the GraphQL API?

In [61]:
author_and_timestamp = sorted(author_and_timestamp, key=lambda x: x[1])
author_and_timestamp

[('Simon Biggs', datetime.datetime(2022, 2, 23, 21, 31, 17)),
 ('Simon Biggs', datetime.datetime(2022, 2, 23, 22, 5, 23)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 2, 38, 31)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 2, 39, 24)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 2, 40, 21)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 3, 58, 30)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 4, 27, 59)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 4, 32, 21)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 4, 34, 6)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 4, 38, 34)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 4, 43, 55)),
 ('GitHub Web Flow', datetime.datetime(2022, 2, 24, 4, 58, 43)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 4, 59, 14)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 4, 59, 41)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 5, 1, 29)),
 ('Simon Biggs', datetime.datetime(2022, 2, 24, 5, 2, 34)),
 ('Simon Biggs', datet

In [66]:
file_creator = author_and_timestamp[0][0]
file_creator

'GitHub Web Flow'

In [67]:
prs_for_this_file = list(commit.get_pulls())
prs_for_this_file

[PullRequest(title="Release sop drc, remove rdm templates for now", number=14)]